In [60]:
from crewai import LLM, Crew, Task, Agent

## **Read LLMs form groq**

In [83]:
import os
from dotenv import load_dotenv
load_dotenv()


os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")



In [84]:
# llm = LLM(model="groq/qwen/qwen3-32b")
llm = LLM(model="groq/meta-llama/llama-4-scout-17b-16e-instruct")

In [85]:
llm

## **Planner agent**

In [86]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on the {topic}.",
    backstory="You are working on planning a blog article about the topics {topic} "
    "you collect info that help the audience learn something and make informed decisions "
    "your work is the basis for the Connect writer to write an article on this topic",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## **Writer Agent**

In [87]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",
    backstory="""
                you are working on a writing a new opinion piece about the topics: {topic}. You base your writing on the work of the content planner, who provides an outline and relevant context about the topic.
                YOu follow the main objectives direction of the outline, as provide by the content planner.
                You also provide objective and information provide by the content planner. You also provide objective and impartial insights and back them up with information provide by the content planner.
                you acknowledge in your opinion piece when you statements are opinions as opposed to objective statements.
    """,
    llm=llm,
    allow_delegation=False,
    verbose=True
    
)

## **Editor Agent**

In [88]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writhing style of the organization. ",
    backstory="""
        You are an editor who receives a blog post from the content writer.
        your goal is to review the blog post to ensure that if follows journalistic best practices, provides balanced viewpoints when providing options or assertions, and also avoids major controversial topics or opinions when possible.
    """,
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## **Task Define**

In [89]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

## **Writer task**

In [90]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

## **Editor Task**

In [91]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew
- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
- verbose=2 allows you to see all the logs of the execution.

In [92]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## **Run the Crew**

In [93]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

┌────────────────────────── Crew Execution Started ───────────────────────────┐
│                                                                             │
│  Crew Execution Started                                                     │
│  Name: crew                                                                 │
│  ID: dbfb970d-c8f4-48ac-8cc9-f61de57cb7d2                                   │
│  Tool Args:                                                                 │
│                                                                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

🚀 Crew: crew
└── 📋 Task: 5484b6a1-9b29-4a9a-b395-671a083f9d45
    Status: Executing Task...
┌───────────────────────────── 🤖 Agent Started ──────────────────────────────┐
│                                                                             │
│  Agent: Content Planner   